In [5]:
import pandas as pd
from main import *
test_data = pd.read_csv("./data/test1_data.csv", index_col=0)

test_data.sample(10)

,text
id,
51,其实半星更合适吧 从情节到演员演技 画面 布景 都是让人分分钟出戏的节奏 国产烂片
594,史上最烂，没有之一。这垃圾片也能上映也是醉了。
890,蛮女大战一堆帅哥。影像追求真实以至于令人昏昏欲睡
609,字幕对不上，没太看懂。好在，安哲的片子有字幕也看不懂；或者，没字幕也看得懂。暂打四星。
675,如果看懂的话，这是一部混合元素的片子，无敌了！！
772,名义上融合了各种中国元素，可我就是笑不起来，感觉有点儿傻逼。
387,导演只为让观众有出乎意料的感觉，不惜挑战常识和极限，情节太怪诞。
274,诗诗啊，我的确觉得你气质完胜杨幂，但选片要谨慎，尤其是作为主角的，这是多烂的片子啊！
832,连爆米花电影都算不上。看得我觉得好累，非常不喜欢女主聒噪的性格，神经质，吵死了。后半段好多了。


In [10]:
import pandas as pd
from main import *


train_data = pd.read_csv('./data/train_data.csv', index_col=0)
label_denotation = {
    1: 'positive',
    0: 'negative'
}
train_data.sample(10)



from sklearn.model_selection import train_test_split

train, val = train_test_split(train_data, test_size=0.17, random_state=12)

databunch = TextClasDataBunch.from_df(".", train, val,
                  tokenizer=fastai_tokenizer,
                  vocab=fastai_bert_vocab,
                  include_bos=False,
                  include_eos=False,
                  text_cols="text",
                  label_cols='label',
                  bs=config.bs,
                  collate_fn=partial(pad_collate, pad_first=False, pad_idx=0),
             )



from pytorch_pretrained_bert.modeling import BertConfig, BertForSequenceClassification
bert_model = BertForSequenceClassification.from_pretrained(config.bert_model_name, num_labels=2)

loss_func = nn.CrossEntropyLoss()

learner = Learner(databunch, bert_model, loss_func=loss_func, metrics=accuracy)
learner.load("./stage1")

Learner(data=TextClasDataBunch;

Train: LabelList (830 items)
x: TextList
[CLS] 故 事 真 的 没 有 一 点 儿 新 的 点 。 编 剧 可 是 邱 刚 健 啊 [UNK] [UNK] 剪 辑 也 挺 二 的 ， 废 戏 太 多 。 [SEP],[CLS] . . . . . . . . . . . . . . 为 毛 石 像 鬼 竟 然 是 天 使 啊 [SEP],[CLS] 史 诗 什 么 的 被 好 莱 坞 盯 上 一 般 都 是 稳 遭 毒 手 ， 一 颓 到 底 的 。 [SEP],[CLS] 为 什 么 那 时 候 的 车 戏 看 起 来 那 么 逼 真 ， 现 在 电 视 剧 里 车 上 的 玻 璃 外 面 明 显 是 贴 的 纸 [SEP],[CLS] 色 彩 饱 满 ， 非 洲 口 音 萌 萌 哒 ， 结 尾 那 个 无 厘 头 的 终 点 根 本 不 是 终 点 好 吗 ！ [SEP]
y: CategoryList
negative,negative,negative,positive,positive
Path: .;

Valid: LabelList (170 items)
x: TextList
[CLS] 经 典 啊 ！ 我 初 中 看 电 视 播 了 片 段 就 记 忆 深 刻 了 [SEP],[CLS] 这 个 故 事 告 诉 我 们 ， 贼 可 以 爱 上 贼 ， 但 贵 妇 却 不 行 ～ [SEP],[CLS] 如 果 不 是 那 个 破 坏 别 人 婚 礼 现 场 的 糟 糕 的 结 尾 ， 这 其 实 是 一 部 还 不 错 的 片 子 。 [SEP],[CLS] 一 场 闹 剧 ， 又 不 好 笑 又 无 营 养 ， 菊 花 变 成 向 日 葵 [SEP],[CLS] 蓝 色 是 忧 郁 的 颜 色 ， 片 子 演 技 有 些 过 了 ， 没 看 完 ， 看 不 下 去 [SEP]
y: CategoryList
positive,positive,positive,negative,negative
Path: .;

Test: None, model=BertForSequenceClassification(
  (bert): 

In [52]:
texts = test_data.text
texts = list(texts)

In [53]:
test_label = []
for text in texts:
    out = learner.predict(text)[2]
    if out[0] > out[1]:
        test_label.append("0")
    else:
        test_label.append("1")

In [57]:
out_f = open("./data/answer.txt", "w")
for label in test_label:
    out_f.write(label)
    out_f.write("\n")
out_f.close()